In [1]:
# Force IPython to auto-flush outputs
%config Application.log_level = 'INFO' 
%config InteractiveShell.ast_node_interactivity = "all"


In [2]:
from langchain_openai import ChatOpenAI
import sys
import os
import io


In [5]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Single LLM Call
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,
    api_key=os.getenv("OPENAI_API_KEY"),  # best practice
    request_timeout=120,
    max_retries=5,
    verbose=True,
   
)

messages = [
    ("assistant", "You are a helpful assistant."),
    ("human", "Write me a haiku about autumn leaves.")
]



res = llm.invoke(messages)   # list-of-conversations -> ChatResult

print(res.content) 

print(res)

Crimson leaves descend,  
Whispers of the cooling breeze,  
Autumn’s soft farewell.
content='Crimson leaves descend,  \nWhispers of the cooling breeze,  \nAutumn’s soft farewell.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 26, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CU7xD1QUqTdD6l6MsJAwfHUK28g9l', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--496cdbc0-11dd-45f1-b20d-aef2eacdc58c-0' usage_metadata={'input_tokens': 26, 'output_tokens': 20, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"J'aime la programmation."

In [8]:
if 1: #this is just a filter to allow or block cell execution
    # Batched LLM Calls
    from langchain_core.messages import HumanMessage, SystemMessage

    messages = [
        ("assistant", "You are a helpful assistant."),
        ([HumanMessage("Write me a haiku about autumn leaves.")]),
    ] * 3

    completion = llm.batch(messages)  # list-of-list-of-conversations -> ChatResult

    print(completion)

    for i, choice in enumerate(completion):
        print(f"\n\nCHOICE {i + 1}\n")
        print(choice.content)


[AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CItZVev8iuk3B0azTCntjYREfl0DF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--455f47d8-6d62-4b2b-a5f9-08000532f5ca-0', usage_metadata={'input_tokens': 18, 'output_tokens': 9, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='Crimson leaves descend,  \nWhispers of the cooling breeze,  \nAutumn’s soft farewell.', additional_kwargs={'refusal': None}, response_metadata={'t

In [ ]:
# Using template allows use insert parameters into the prompt - you only need to provide parameter values at query time
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "French",
        "input": "I love programming.",
    }
)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)

# --- First prompt: reformulate ---
reformulate_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that rewrites questions in a clearer, more formal way."),
    ("user", "Rewrite the following question to be clear and formal: {raw_question}")
])

# --- Second prompt: answer the reformulated question ---
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a knowledgeable tutor."),
    ("user", "Answer this question in a concise and friendly way:\n\n{clean_question}")
])

# --- Chain them together ---
# reformulate → llm → inject into second prompt → llm
chain = (
    reformulate_prompt 
    | llm 
    | (lambda msg: {"clean_question": msg.content})  # extract content
    | answer_prompt 
    | llm
)

# --- Run the chain ---
query = "hey what is chatprompttemplate used for???"
response = chain.invoke({"raw_question": query})

print("\n Input prompt:", query)
print(response.content)

# Run only the first prompt + LLM
first_result = (reformulate_prompt | llm).invoke({"raw_question": query})
print("\n Before reformulation:", query)
print("\n After reformulation:", first_result.content)

second_result = (answer_prompt | llm).invoke({"clean_question": first_result.content})
print("\n After answering:", second_result.content)

In [ ]:
if False:

    # how to have an interactive real time conversation with the assistant?

    # Initialise model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

    def ask_once(user_input: str):
        # Only system + user, no history
        messages = [
            SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content=user_input)
        ]
        response = llm.invoke(messages)
        return response.content

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = ask_once(user_input)
        i += 1
        print(f"Assistant: {answer}\n")

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?

What did you notice about this Assistant?

In [ ]:
if False:

    # What is diffeernt about this version?

    # initialise model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

    # start with a system prompt
    history = [SystemMessage(content="You are a helpful assistant.")]

    # function to ask user input and continue the conversation
    def chat_turn(user_input: str):
        # append user message
        history.append(HumanMessage(content=user_input))

        # call model with full history
        ai_msg = llm.invoke(history)  # returns an AIMessage
        #print(f"Assistant: {ai_msg.content}")

        # append assistant reply manually (so it's fed into the next turn)
        history.append(ai_msg)
        return ai_msg.content

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = chat_turn(user_input)
        i += 1
        print(f"Assistant: {answer}\n")

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?

In [12]:
if 1:
    # And about this version?

    # initialise model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0,output_version="responses/v1")

    tool = {"type": "web_search_preview"}
    llm_with_tools = llm.bind_tools([tool])

    # start with a system prompt
    history = [SystemMessage(content="You are a helpful assistant.")]

    # function to ask user input and continue the conversation
    def chat_turn(user_input: str):
        # append user message
        history.append(HumanMessage(content=user_input))

        # call model with full history
        ai_msg = llm_with_tools.invoke(history)  # returns an AIMessage
        #print(f"Assistant: {ai_msg.content}")

        # append assistant reply manually (so it's fed into the next turn)
        history.append(ai_msg)
        return ai_msg.text()

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = chat_turn(user_input)
        i += 1
        print(f"Assistant: {answer}\n")
        print({answer})

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?
        # who won the women's 100m  hurldes at the 2025 tokyo world athletics championship

Assistant: As of September 2025, the Prime Minister of the United Kingdom is Sir Keir Starmer. He assumed office on 5 July 2024, following the Labour Party's victory in the general election. ([gov.uk](https://www.gov.uk/government/people/keir-starmer?utm_source=openai)) Prior to his premiership, Starmer served as the Leader of the Opposition from April 2020 to July 2024. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Keir_Starmer?utm_source=openai))

In September 2025, Starmer appointed David Lammy as Deputy Prime Minister and Secretary of State for Justice. ([en.wikipedia.org](https://en.wikipedia.org/wiki/David_Lammy?utm_source=openai))


## Recent Developments in UK Politics:
- [UK's Starmer to press ahead with digital ID plans, FT reports](https://www.reuters.com/world/uk/uks-starmer-set-push-digital-id-plans-ft-reports-2025-09-19/?utm_source=openai)
- [Trump begins historic state visit to UK amid pomp and protests](https://www.reuters.com/world/uk/trump-begins-historic-state-vi

In [ ]:
print(history)

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel


class WeatherOutput(BaseModel):
    answer: str
    justification: str

def get_weather(location: str) -> WeatherOutput: # Here note that unlike inbuilt tools like web_search which OpenAI run server side, these function tools run client side and so currently langchain doesn't automatically run them for us
    """Gets the weather at a location"""
    return WeatherOutput(
        answer= f"It's sunny in {location}.",
        justification="Mocking weather API that always returns sunny for demo."
    )

class OutputSchema(BaseModel):
    """Schema for response."""

    answer: str
    justification: str

llm = ChatOpenAI(model="gpt-4.1")

structured_llm = llm.bind_tools(
    [get_weather],)
    
#     response_format=OutputSchema,
#     strict=True,
# )

# Response contains tool calls:
tool_call_response = structured_llm.invoke("What is the weather in San Francisco?")
print("Weather response", tool_call_response.content)
print("Weather response tool calls", tool_call_response.tool_calls)
# structured_response.additional_kwargs["parsed"] contains parsed output
structured_response = structured_llm.invoke(
    "What weighs more, a pound of feathers or a pound of gold?"
)
print("Puzzle response", structured_response.content)

Weather response 
Weather response tool calls [{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_vEseQaMcTOjXAwdAzOXLxP1L', 'type': 'tool_call'}]
Puzzle response A pound of feathers and a pound of gold both weigh the same—they each weigh one pound. 

However, there is an interesting detail: in the past, gold and other precious metals were sometimes weighed using a different system called the troy pound, which is slightly lighter than the common avoirdupois pound used for most other items. 

- 1 avoirdupois pound = 16 ounces = 453.6 grams (used for feathers)
- 1 troy pound = 12 ounces = 373.2 grams (used for gold)

So if you use these traditional systems, a "pound" of feathers (avoirdupois) would actually weigh more than a "pound" of gold (troy). But if you’re simply talking about one pound (without specifying systems), they weigh the same.


In [65]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from pydantic import BaseModel


class WeatherOutput(BaseModel):
    answer: str
    justification: str

@tool #I tried using the tool decorator but it made no difference
def get_weather(location: str) -> WeatherOutput: # Here note that unlike inbuilt tools like web_search which OpenAI run server side, these function tools run client side and so currently langchain doesn't automatically run them for us
    """Gets the weather at a location"""
    return WeatherOutput(
        answer= f"It's sunny in {location}.",
        justification="Mocking weather API that always returns sunny for demo."
     )

class OutputSchema(BaseModel):
    """Schema for response."""

    answer: str
    justification: str

llm = ChatOpenAI(model="gpt-4.1")

structured_llm = llm.bind_tools(
    [get_weather],
    
    response_format=OutputSchema,
    strict=True,
)

# Response contains tool calls:
tool_call_response = structured_llm.invoke("What is the weather in San Francisco?")
print(tool_call_response)
print("Weather response", tool_call_response.content)
print("Weather response tool calls", tool_call_response.tool_calls)
# structured_response.additional_kwargs["parsed"] contains parsed output
structured_response = structured_llm.invoke(
    "What weighs more, a pound of feathers or a pound of gold?"
)
print("\n",structured_response)
print("Puzzle response", structured_response.content)

content='' additional_kwargs={'tool_calls': [{'id': 'call_uSiXJrCzpWzCqU9Od2cpbIHc', 'function': {'arguments': '{"location":"San Francisco"}', 'name': 'get_weather', 'parsed_arguments': {'location': 'San Francisco'}}, 'type': 'function'}], 'parsed': None, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 103, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_62bb2e3c55', 'id': 'chatcmpl-CIvT7FFyWwryj5XDuHZd8R6lpLPOW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--c01b8409-6f2b-4fbb-9265-0caa0df1ce95-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_uSiXJrCzpWzCqU9Od2cpbIHc', 'type': 'tool_call'}] usage_metadata={'input_tokens': 10

In [49]:
from importlib.metadata import version
print(version("langchain-openai"))


0.3.33


In [ ]:
from openai import OpenAI #just trying out the OpenAI client. OpenAI client is not as friendly as langchain but it works
client = OpenAI()

# Example of a function tool call
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather for a city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"],
            },
        },
    }
]

resp = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "user", "content": "What's the weather in London?"}],
    tools=tools,
    tool_choice="auto",
)

print(resp.choices[0].message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_0jN7uAcaQH5wD2ZYvEFxtV0H', function=Function(arguments='{"city":"London"}', name='get_weather'), type='function')])


## Embeddings

In [73]:
# Imports
from langchain_openai import OpenAIEmbeddings
import numpy as np

# Initialize the embeddings client
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=3072)


In [74]:
# Helper: get an embedding as a NumPy array
def embed(text: str) -> np.ndarray:
    vec = embeddings.embed_query(text)   # returns a Python list of floats
    return np.array(vec)


In [78]:
# Generate embeddings for the words of interest
king = embed("king")
queen = embed("queen")
man = embed("man")
woman = embed("woman")

Paris = embed("Paris")
France = embed("France")
Italy = embed("Italy")
Rome= embed("Rome")


In [76]:
# Word math: queen ≈ woman + king − man
approx_queen = woman + king - man

# Compare cosine similarity
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarity = cosine_similarity(queen, approx_queen)
print(f"Cosine similarity between true queen and computed vector: {similarity:.4f}")


Cosine similarity between true queen and computed vector: 0.5214


In [77]:
# Just for illustration: rank nearest among our 4 words
vectors = {"king": king, "queen": queen, "man": man, "woman": woman}
scores = {word: cosine_similarity(approx_queen, vec) for word, vec in vectors.items()}
sorted(scores.items(), key=lambda x: x[1], reverse=True)


[('king', np.float64(0.6903742809749485)),
 ('woman', np.float64(0.5696693490830003)),
 ('queen', np.float64(0.5213563243282194)),
 ('man', np.float64(-0.008197622219882548))]

In [79]:
approx_Rome= France + Rome - Italy
print(cosine_similarity(Rome, approx_Rome))

0.6983634582040057


In [ ]:
#Example 1: Semantic similarity of paraphrases
sents = [
    "A man is playing a guitar.",
    "Someone is strumming an instrument.",
    "The weather is sunny and warm.",
    "A woman is cooking dinner."
]

vecs = {s: embed(s) for s in sents}
target = vecs["A man is playing a guitar."]

sims = [(s, cosine_similarity(target, v)) for s,v in vecs.items()]
sorted(sims, key=lambda x: -x[1])


[('A man is playing a guitar.', np.float64(0.9999999999999999)),
 ('Someone is strumming an instrument.', np.float64(0.5117938139436554)),
 ('A woman is cooking dinner.', np.float64(0.2242725686883578)),
 ('The weather is sunny and warm.', np.float64(0.14281628743630567))]

In [85]:
#Example 2: Topic grouping
sports = [
    "The football match went into extra time.",
    "The football match ended in a draw.",
    "Arsenal won 3-2.",
    "Arsenal beat Liverpool 3-2.",
    "A basketball player scored a triple-double."
]
science = [
    "The experiment confirmed the hypothesis.",
    "Researchers discovered a new species of frog."
]
all_sents = sports + science

vecs = {s: embed(s) for s in all_sents}
target = embed("Who won the soccer game yesterday?")

sims = [(s, cosine_similarity(target, v)) for s,v in vecs.items()]
sorted(sims, key=lambda x: -x[1])


[('The football match ended in a draw.', np.float64(0.40843571805118306)),
 ('Arsenal won 3-2.', np.float64(0.38845099768817515)),
 ('Arsenal beat Liverpool 3-2.', np.float64(0.3652957204670271)),
 ('The football match went into extra time.', np.float64(0.3592746323447075)),
 ('A basketball player scored a triple-double.',
  np.float64(0.2747090663570589)),
 ('The experiment confirmed the hypothesis.', np.float64(0.12043236856133151)),
 ('Researchers discovered a new species of frog.',
  np.float64(0.07104991386677663))]

In [92]:
# Example 4: Document search toy demo
docs = [
    "The Mona Lisa is a famous painting by Leonardo da Vinci.",
    "Mount Everest is the tallest mountain in the world.",
    "The Grand Canyon is the lowest trough in the world. ",
    "Chinese is the most spoken language in the world.",
    "Python is a popular programming language for machine learning."
]

vecs = {d: embed(d) for d in docs}
query = "Which mountain is the highest on Earth?"
qvec = embed(query)

sims = [(d, cosine_similarity(qvec, v)) for d,v in vecs.items()]
sorted(sims, key=lambda x: x[1], reverse=True)


[('Mount Everest is the tallest mountain in the world.',
  np.float64(0.5875616182838077)),
 ('The Grand Canyon is the lowest trough in the world. ',
  np.float64(0.24209245155821002)),
 ('Chinese is the most spoken language in the world.',
  np.float64(0.1528612564098758)),
 ('The Mona Lisa is a famous painting by Leonardo da Vinci.',
  np.float64(0.09994392241515195)),
 ('Python is a popular programming language for machine learning.',
  np.float64(0.05047505054960984))]

In [94]:
s1 = "Paris is the capital of France."
s2 = "Berlin is the capital of Germany."
s3 = "Ottawa is the capital of Canada."
s4 = "Europe."

vec_math = embed(s1) - embed(s4) + embed(s3)

# Compare to candidate sentences
candidates = [
    "Paris is the capital of France.",
    "Berlin is the capital of Germany.",
    "Ottawa is the capital of Canada.",
    "Tokyo is the capital of Japan."
]

vecs = {c: embed(c) for c in candidates}
scores = {c: cosine_similarity(vec_math, v) for c,v in vecs.items()}
sorted(scores.items(), key=lambda x: -x[1])


[('Ottawa is the capital of Canada.', np.float64(0.7563769556955102)),
 ('Paris is the capital of France.', np.float64(0.6828128201082201)),
 ('Tokyo is the capital of Japan.', np.float64(0.5335522161467855)),
 ('Berlin is the capital of Germany.', np.float64(0.43951000420692404))]